In [1]:
# Parameters
data_url = "https://osf.io/ydxt7/"


In [2]:
import endomill
from nbmetalog import nbmetalog as nbm


In [3]:
nbm.print_metadata()


context: ci
hostname: 86e68033af0d
interpreter: 3.8.12 (default, Jan 15 2022, 18:39:47)  [GCC 7.5.0]
nbcellexec: 3
nbname: phylogeny_simulation_mill
nbpath: /opt/hereditary-stratigraph-concept/binder/phylogenetic-inference/phylogeny_simulation_mill.ipynb
revision: null
session: 5bf2c42c-89bc-4434-a2e6-30b812f14edd
timestamp: 2022-03-06T23:53:24Z00:00


IPython==7.16.1
keyname==0.4.1
yaml==5.3.1
endomill==0.1.2
nbmetalog==0.2.6
re==2.2.1
ipython_genutils==0.2.0
logging==0.5.1.2
zmq==22.3.0
json==2.0.9
ipykernel==5.5.3


In [4]:
endomill.instantiate_over(
    parameter_packs=[
        {'data_url': data_url}
        for data_url in [
            'https://osf.io/cz9fk/',
            'https://osf.io/ydxt7/',
            'https://osf.io/8ycq7/',
            'https://osf.io/5ubn8/',
        ]
    ],
)


detected executing.endomill.ipynb file
skipping instantiate_over


In [5]:
# define papermill parameters
data_url: str


In [6]:
import anytree
from bs4 import BeautifulSoup
from collections import defaultdict
from hstrat import hstrat
from interval_search import doubling_search
from iterpop import iterpop as ip
import itertools as it
from keyname import keyname as kn
import opytional as opyt
import pandas as pd
import random
import sys
from tqdm import tqdm
from urllib import request


In [7]:
random.seed(1) # ensure reproducibility
sys.setrecursionlimit(100000) # anytree exceeds default recursion limit on our data


In [8]:
nbm.print_metadata()


context: ci
hostname: 86e68033af0d
interpreter: 3.8.12 (default, Jan 15 2022, 18:39:47)  [GCC 7.5.0]
nbcellexec: 8
nbname: phylogeny_simulation_mill
nbpath: /opt/hereditary-stratigraph-concept/binder/phylogenetic-inference/phylogeny_simulation_mill.ipynb
revision: null
session: 5bf2c42c-89bc-4434-a2e6-30b812f14edd
timestamp: 2022-03-06T23:53:25Z00:00


IPython==7.16.1
endomill==0.1.2
ipykernel==5.5.3
ipython_genutils==0.2.0
json==2.0.9
keyname==0.4.1
logging==0.5.1.2
nbmetalog==0.2.6
re==2.2.1
yaml==5.3.1
zmq==22.3.0
anytree==2.8.0
hstrat==0.2.0
iterpop==0.3.4
opytional==0.1.0
pandas==1.1.2


In [9]:
html = request.urlopen(data_url).read().decode('utf8')

soup = BeautifulSoup(html, 'html.parser')
title = soup.find('title')

data_filename = title.string.split()[-1]
print(data_filename)


nk_randomselection_seed7_pop100_mut.01_snapshot_5000.csv


In [10]:
endomill.add_instance_outpath(
    f'a=phylogeny_simulation+source={data_filename}.endomill.ipynb'
)


# Retrieve Target Phylogeny from OSF


In [11]:
target_phylogeny_df = pd.read_csv(
    f'{data_url}/download',
)

nbm.print_dataframe_synopsis(target_phylogeny_df)


digest: 542298015c00bd09372befa775965be67fd83bbdb47a940347c178f5a19083d0
manifest:
  ancestor_list: '    3626#  ex., [316479]'
  depth: '            3153#  ex., 3141'
  destruction_time: ' 2743#  ex., inf'
  id: '               3697#  ex., 316633'
  num_offspring: '    5#     ex., 0'
  num_orgs: '         5#     ex., 1'
  origin_time: '      3211#  ex., 5000'
  phenotype: '        3686#  ex., [ 0.797946 0.536379 0.443815 0.0526468 0.92057 0.91075
    0.30778 0.492666 0.443872 0.822448 0.850668 0.622605 0.862708 0.278087 0.28528
    0.595257 0.709313 0.589555 0.0954585 0.862811 0.681532 0.396538 0.940159 0.466114
    0.964965 0.496943 0.931246 0.459464 0.323874 0.381974 0.457127 0.807726 0.439315
    0.824859 0.57682 0.840605 0.720875 0.501633 0.541537 0.474429 0.260919 0.477315
    0.382976 0.773835 0.691462 0.0166064 0.756788 0.431525 0.341946 0.359509 0.0311168
    0.624921 0.449223 0.775889 0.485843 0.932592 0.949503 0.0422867 0.0536213 0.108818
    0.958998 0.540366 0.456285 0.1875

# Create a Tree with Target Phylogeny Structure


In [12]:
# map id to anytree node
nodes = defaultdict(anytree.AnyNode)
for __, row in target_phylogeny_df.iterrows():
    node = nodes[row['id']]
    node.id = row['id']
    node.origin_time = row['origin_time']
    if 'NONE' not in row['ancestor_list']:
        ancestor_id = ip.popsingleton(
            eval(row['ancestor_list'])
        )
        node.parent = nodes[ancestor_id]

roots = {node.root for node in nodes.values()}


In [13]:
root = ip.popsingleton(roots)
root.height


3152

# Pick Parameters for Hereditary Stratigraphic Columns


In [14]:
def make_conditions(num_generations: int) -> pd.DataFrame:
    res = []
    for condemner_factory, target_column_bits, differentia_bit_width in it.product(
        [
            hstrat.StratumRetentionCondemnerTaperedDepthProportionalResolution,
            hstrat.StratumRetentionCondemnerRecencyProportionalResolution,
        ],
        [
            64,
            64 * 8,
            64 * 16,
        ],
        [
            1,
            8,
            64,
        ],
    ):
        policy_param = doubling_search(
            lambda x: \
                condemner_factory(x + 1).CalcNumStrataRetainedExact(num_generations)
                * differentia_bit_width > target_column_bits or x >= num_generations,
            {
                hstrat.StratumRetentionCondemnerTaperedDepthProportionalResolution: 1,
                hstrat.StratumRetentionCondemnerRecencyProportionalResolution: 0,
            }[condemner_factory],
        )

        actual_column_strata = condemner_factory(policy_param).CalcNumStrataRetainedExact(num_generations)
        actual_column_bits = actual_column_strata * differentia_bit_width

        res.append({
            'Retention Policy' : condemner_factory.__name__[25:],
            'Differentia Bit Width' : differentia_bit_width,
            'Retention Policy Resolution Parameter' : policy_param,
            'Target Retained Bits' : target_column_bits,
            'Actual Retained Bits' : actual_column_bits,
            'Retained Bits Error' : actual_column_bits - target_column_bits,
            'Actual Retained Strata' : actual_column_strata,
            'condemner' : condemner_factory(policy_param),
        })
    return pd.DataFrame.from_records(res)


In [15]:
conditions_df = make_conditions(root.height + 1)
conditions_df.drop('condemner', axis=1)


,Retention Policy,Differentia Bit Width,Retention Policy Resolution Parameter,Target Retained Bits,Actual Retained Bits,Retained Bits Error,Actual Retained Strata
0,TaperedDepthProportionalResolution,1,31,64,63,-1,63
1,TaperedDepthProportionalResolution,8,3,64,56,-8,7
2,TaperedDepthProportionalResolution,64,1,64,192,128,3
3,TaperedDepthProportionalResolution,1,256,512,512,0,512
4,TaperedDepthProportionalResolution,8,31,512,504,-8,63
5,TaperedDepthProportionalResolution,64,3,512,448,-64,7
6,TaperedDepthProportionalResolution,1,512,1024,1024,0,1024
7,TaperedDepthProportionalResolution,8,64,1024,1024,0,128
8,TaperedDepthProportionalResolution,64,7,1024,960,-64,15
9,RecencyProportionalResolution,1,6,64,63,-1,63


# Set Up Ancestor Column


In [16]:
bundle = hstrat.HereditaryStratigraphicColumnBundle({
    kn.pack({
        'differentia' : row['Differentia Bit Width'],
        'policy' : row['Retention Policy'],
        'resolution' : row['Retention Policy Resolution Parameter'],
        'target_bits' : row['Target Retained Bits'],
        'actual_bits' : row['Actual Retained Bits'],
        'bits_error' : row['Retained Bits Error'],
        'actual_strata' : row['Actual Retained Strata'],
    }) \
        : hstrat.HereditaryStratigraphicColumn(
            stratum_differentia_bit_width=row['Differentia Bit Width'],
            stratum_retention_condemner=row['condemner'],
    )
    for __, row in conditions_df.iterrows()
})


# Simulate Inheritance of Ancestor Column Down Phylogenetic Tree


In [17]:
root = ip.popsingleton(roots)
root.hstrat_column = bundle

for node in anytree.LevelOrderIter(ip.popsingleton(roots)):
    parent = node.parent
    if parent is not None:
        node.hstrat_column = parent.hstrat_column.Clone()
        for __ in range(node.origin_time - parent.origin_time): node.hstrat_column.DepositStratum()


# Extract Pairwise MRCA Estimates for Extant Organisms


In [18]:
res = []
for extant1, extant2 in tqdm([*it.product(root.leaves, root.leaves)]):
    if extant1 != extant2:
        bounds = extant1.hstrat_column.CalcRankOfMrcaBoundsWith(extant2.hstrat_column)
        for impl in extant1.hstrat_column:
            res.append({
                'Column Configuration' \
                    : impl,
                'Differentia Bit Width' \
                    : kn.unpack(impl)['differentia'],
                'Stratum Retention Policy' \
                    : kn.unpack(impl)['policy'],
                'Stratum Retention Policy Resolution Parameter' \
                    : kn.unpack(impl)['resolution'],
                'Stratigraphic Column Actual Retained Bits' \
                    : kn.unpack(impl)['actual_bits'],
                'Stratigraphic Column Target Retained Bits' \
                    : kn.unpack(impl)['target_bits'],
                'Stratigraphic Column Retained Bits Error' \
                    : kn.unpack(impl)['bits_error'],
                'Stratigraphic Column Actual Num Retained Strata' \
                    : kn.unpack(impl)['actual_strata'],
                'Taxon Compared From' \
                    : extant1.id,
                'Taxon Compared To' \
                    : extant2.id,
                'Generation of Taxon Compared From' \
                    : extant1.hstrat_column.GetNumStrataDeposited(),
                'Generation of Taxon Compared To' \
                    : extant2.hstrat_column.GetNumStrataDeposited(),
                'Generation Of MRCA Lower Bound (inclusive)' \
                    : opyt.apply_if(
                        bounds[impl],
                        lambda x: x[0],
                    ),
                'Generation Of MRCA Upper Bound (exclusive)' \
                    : opyt.apply_if(
                        bounds[impl],
                        lambda x: x[1],
                    ),
                'MRCA Bound Confidence' \
                    : extant1.hstrat_column[impl].CalcRankOfMrcaBoundsWithProvidedConfidenceLevel(),
                'Rank of Earliest Detectable Mrca With' \
                    : extant1.hstrat_column[impl].CalcRankOfEarliestDetectableMrcaWith(extant2.hstrat_column[impl]),
            })

res_df = pd.DataFrame.from_records(res)


  0%|          | 0/5184 [00:00<?, ?it/s]

  0%|          | 5/5184 [00:00<01:59, 43.30it/s]

  0%|          | 9/5184 [00:00<02:06, 40.96it/s]

  0%|          | 13/5184 [00:00<02:10, 39.70it/s]

  0%|          | 17/5184 [00:00<02:12, 38.92it/s]

...

100%|█████████▉| 5169/5184 [02:10<00:00, 40.71it/s]

100%|█████████▉| 5174/5184 [02:10<00:00, 39.63it/s]

100%|█████████▉| 5178/5184 [02:10<00:00, 38.84it/s]

100%|█████████▉| 5182/5184 [02:10<00:00, 38.16it/s]

100%|██████████| 5184/5184 [02:10<00:00, 39.59it/s]

In [19]:
res_df


,Column Configuration,Differentia Bit Width,Stratum Retention Policy,Stratum Retention Policy Resolution Parameter,Stratigraphic Column Actual Retained Bits,Stratigraphic Column Target Retained Bits,Stratigraphic Column Retained Bits Error,Stratigraphic Column Actual Num Retained Strata,Taxon Compared From,Taxon Compared To,Generation of Taxon Compared From,Generation of Taxon Compared To,Generation Of MRCA Lower Bound (inclusive),Generation Of MRCA Upper Bound (exclusive),MRCA Bound Confidence,Rank of Earliest Detectable Mrca With
0,actual_bits=63+actual_strata=63+bits_error=-1+...,1,TaperedDepthProportionalResolution,31,63,64,-1,63,316552,316596,5000,5001,4736,5000,0.968750,512
1,actual_bits=56+actual_strata=7+bits_error=-8+d...,8,TaperedDepthProportionalResolution,3,56,64,-8,7,316552,316596,5000,5001,4608,5000,0.996094,0
2,actual_bits=192+actual_strata=3+bits_error=128...,64,TaperedDepthProportionalResolution,1,192,64,128,3,316552,316596,5000,5001,4096,5000,1.000000,0
3,actual_bits=512+actual_strata=512+bits_error=0...,1,TaperedDepthProportionalResolution,256,512,512,0,512,316552,316596,5000,5001,4960,5000,0.968750,64
4,actual_bits=504+actual_strata=63+bits_error=-8...,8,TaperedDepthProportionalResolution,31,504,512,-8,63,316552,316596,5000,5001,4992,5000,0.996094,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92011,actual_bits=504+actual_strata=63+bits_error=-8...,8,RecencyProportionalResolution,6,504,512,-8,63,316622,316382,5001,4998,4992,4993,0.996094,0
92012,actual_bits=320+actual_strata=5+bits_error=-19...,64,RecencyProportionalResolution,0,320,512,-192,5,316622,316382,5001,4998,4992,4998,1.000000,0
92013,actual_bits=1022+actual_strata=1022+bits_error...,1,RecencyProportionalResolution,209,1022,1024,-2,1022,316622,316382,5001,4998,4992,4997,0.968750,64
92014,actual_bits=1000+actual_strata=125+bits_error=...,8,RecencyProportionalResolution,14,1000,1024,-24,125,316622,316382,5001,4998,4992,4993,0.996094,0


In [20]:
nbm.print_dataframe_synopsis(res_df)


digest: 667ec563d5dfe166609e43adfaec0261082fa38af9b3ab2ad0731de62c85b836
manifest:
  Column Configuration: '                            18#  ex., actual_bits=63+actual_strata=63+bits_error=-1+differentia=1+policy=TaperedDepthProportionalResolution+resolution=31+target_bits=64'
  Differentia Bit Width: '                           3#   ex., 1'
  Generation Of MRCA Lower Bound (inclusive): '      66#  ex., 4736'
  Generation Of MRCA Upper Bound (exclusive): '      46#  ex., 5000'
  Generation of Taxon Compared From: '               4#   ex., 5000'
  Generation of Taxon Compared To: '                 4#   ex., 5001'
  MRCA Bound Confidence: '                           3#   ex., 0.96875'
  Rank of Earliest Detectable Mrca With: '           6#   ex., 512'
  Stratigraphic Column Actual Num Retained Strata: ' 12#  ex., 63'
  Stratigraphic Column Actual Retained Bits: '       13#  ex., 63'
  Stratigraphic Column Retained Bits Error: '        9#   ex., -1'
  Stratigraphic Column Target Retained 

# Save Pairwise MRCA Estimates to File


In [21]:
res_df.to_csv(
    f'a=pairwise_mrca_estimates+source={data_filename}.gz',
    compression='gzip',
)
